In [ ]:
# load the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import plotly.express as px
import plotly.graph_objects as go
import gc

In [ ]:
# read data
data_all = pd.read_csv('gimnasio.csv')

In [ ]:
data_all.head()

In [ ]:
target = 'Intensity_gimnasio'

In [ ]:
data=data_all[{target}].dropna()

In [ ]:
# plot the data
px.scatter(x = data.index, y=data[target],
       labels={'x':'Number of time steps (days)',
               'y':f'{target}'})

In [ ]:
# scale the data
from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler(feature_range=(0,1))
data = scale.fit_transform(data)

In [ ]:
# divide the data into number of steps 
def create_data(data, seq_len):
    N = len(data)
    X = []
    Y = []
    for i in range(N-seq_len-1):
        x = data[i:i+seq_len]
        X.append(x)
        y = data[i+seq_len]
        Y.append(y)        
    return X,Y

In [ ]:
x, y = create_data(data,10)
# convert the x and y values into arrays
x, y = np.asarray(x), np.asarray(y)
x.reshape(-1,10).shape

In [ ]:
# divide the data into train and test
def split_data(x,y, ratio):
    assert len(x)==len(y)
    N = len(x)
    train_x, test_x  = x[:int(N*ratio)], x[int(N*ratio):]
    train_y, test_y  = y[:int(N*ratio)], y[int(N*ratio):]
    return train_x,train_y, test_x, test_y

### Recurrent Neural Network - RNN

In [ ]:
# Implement the RNN 
class Simple_RNN(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim, num_layers):
        super().__init__()
        
        # define the dimensions
        self.in_dim = in_dim
        self.hid_dim = in_dim
        self.out_dim = out_dim
        self.layer = num_layers
        
        # define the rnn layer
        self.rnn = nn.RNN(self.in_dim, self.hid_dim, self.layer, nonlinearity='tanh', batch_first=True)
        
        # define fully connected layer for output
        self.fc = nn.Linear(self.hid_dim, self.out_dim)
        
    def forward(self,x):
        
        # initialize the hidden layer
        h0 = torch.zeros(self.layer, x.size(0),self.hid_dim)
        
        #initialize the rnn
        out, _ = self.rnn(x,h0)
        
        out = self.fc(out[:,-1,:])
        return out

In [ ]:
# convert the data from numpy to tensor
train_x, train_y, test_x, test_y = split_data(x, y, 0.7)
train_x = torch.from_numpy(train_x.astype(np.float32))
train_y = torch.from_numpy(train_y.astype(np.float32))
test_x = torch.from_numpy(test_x.astype(np.float32))
test_y = torch.from_numpy(test_y.astype(np.float32))

In [ ]:
x.shape

In [ ]:
# Parameters of RNN neural network
in_size = x.shape[-1]
out_size = in_size
hid_size = 3
num_layers = 1

In [ ]:
# _______________MODEL___________________________________
model = Simple_RNN(in_size,
                   hid_size,
                   out_size, 
                   num_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fun = nn.MSELoss()
epochs = 500

In [ ]:
def train_model(model,
               criterion,
               optimizer,
               x_train,
               x_test,
               y_train,
               y_test,
               epochs=epochs):   
    train_loss = np.zeros(epochs)
    test_loss = np.zeros(epochs)    
    for epoch in range(epochs):        
        optimizer.zero_grad() # put default model grads to zero
        
        # predict the output
        pred = model(x_train)
        
        # calculate the loss 
        error = criterion(pred,y_train)
        
        # backpropagate the error
        error.backward()
        
        # update the model parameters
        optimizer.step()
        
        # save the losses 
        train_loss[epoch] = error.item()
        
        # test loss 
        test_pred = model(x_test)
        test_error = criterion(y_test,test_pred)
        test_loss[epoch] = test_error.item()
        
        if (epoch+1) % 5 ==0:
            print('Epoch :{}    Train Loss :{}    Test Loss :{}'.format((epoch+1)/epochs, error.item(), test_error.item()))
            
    return train_loss, test_loss

In [ ]:
train_loss, test_loss = train_model(model,
               loss_fun,
               optimizer,
               train_x,
               test_x,
               train_y,
               test_y,
               epochs=500)

In [ ]:
import plotly.graph_objects as go
trace11 = go.Scatter(
    x = np.arange(0, epochs,1),
    y = train_loss,
    mode = 'lines',
    name = 'Train Loss'
)
trace21 = go.Scatter(
    x = np.arange(0, epochs,1),
    y = test_loss,
    mode = 'lines',
    name = 'Test Loss'
)

layout = go.Layout(
    title = f"RNN performace",
    xaxis = {'title' : 'Epochs'},
    yaxis = {'title' : 'Loss'}
)
fig = go.Figure(data=[trace11, trace21], layout=layout)
fig.show()

In [ ]:
# testing the prediction quality if the trained model

trace31 = go.Scatter(
    y = np.vstack((train_y.detach().numpy(),test_y.detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Target'
)
trace41 = go.Scatter(
    y = np.vstack((train_y.detach().numpy(),model(test_x).detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Prediction'
)

layout = go.Layout(
    title = f"Evolution of {target}",
    xaxis = {'title' : 'Time delta in days'},
    yaxis = {'title' : f'{target}'}
)
fig = go.Figure(data=[trace31, trace41], layout=layout)
fig.show()

### LSTM

In [ ]:
# Implement LSTM
class LSTM(nn.Module):
    def __init__(self,in_dim, hid_dim, out_dim, num_layers):
        super().__init__()
        self.in_dim = in_dim
        self.hid_dim = hid_dim
        self.out_dim = out_dim
        self.layer = num_layers       
        self.lstm = nn.LSTM(self.in_dim, self.hid_dim, self.layer, batch_first=True)        
        self.fc = nn.Linear(self.hid_dim, self.out_dim)
        
    def forward(self,x):        
        # initialize the hidden state
        h0 = torch.zeros(self.layer, x.size(0), self.hid_dim)
        c0 = torch.zeros(self.layer, x.size(0), self.hid_dim)
        
        # forward propagate
        out, (h_,c_) = self.lstm(x,(h0,c0))
        
        out = self.fc(out[:,-1,:])
        return out

In [ ]:
x, y = create_data(data,10)
# convert the x and y values into arrays
x, y = np.asarray(x), np.asarray(y)
x.reshape(-1,10).shape

In [ ]:
# Define LSTM RNN neural network
in_size = x.shape[-1]
out_size = in_size
hid_size = 5
num_layers = 1
lstm = LSTM(in_size,hid_size,out_size, num_layers)
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.01)
loss_fun = nn.MSELoss()

In [ ]:
# convert the data from numpy to tensor
train_x1, train_y1, test_x1, test_y1 = split_data(x, y, 0.7)
train_x1 = torch.from_numpy(train_x1.astype(np.float32))
train_y1 = torch.from_numpy(train_y1.astype(np.float32))
test_x1 = torch.from_numpy(test_x1.astype(np.float32))
test_y1 = torch.from_numpy(test_y1.astype(np.float32))

In [ ]:
x.shape

In [ ]:
epochs1 = 500
train_loss1, test_loss1 = train_model(lstm,
               loss_fun,
               optimizer,
               train_x1,
               test_x1,
               train_y1,
               test_y1,
               epochs=epochs1)

In [ ]:
import plotly.graph_objects as go
trace12 = go.Scatter(
    x = np.arange(0, epochs1,1),
    y = train_loss1,
    mode = 'lines',
    name = 'Train Loss'
)
trace22 = go.Scatter(
    x = np.arange(0, epochs1,1),
    y = test_loss1,
    mode = 'lines',
    name = 'Test Loss'
)

layout = go.Layout(
    title = f"RNN performace",
    xaxis = {'title' : 'Epochs'},
    yaxis = {'title' : 'Loss'}
)
fig = go.Figure(data=[trace12, trace22], layout=layout)
fig.show()

In [ ]:
# testing the prediction quality if the trained model

trace32 = go.Scatter(
    y = np.vstack((train_y1.detach().numpy(),test_y1.detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Target'
)
trace42 = go.Scatter(
    y = np.vstack((train_y1.detach().numpy(),lstm(test_x1).detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Prediction'
)

layout = go.Layout(
    title = f"Evolution of feature '{target}' with time",
    xaxis = {'title' : 'Time delta in days'},
    yaxis = {'title' : f'{target}'}
)
fig = go.Figure(data=[trace32, trace42], layout=layout)
fig.show()

### Multi-layer gated recurrent unit RNN -  GRU

In [ ]:
# implement GRU
class GRU(nn.Module):
    def __init__(self,in_dim, hid_dim, out_dim, num_layer):
        super().__init__()
        
        self.in_dim = in_dim
        self.hid_dim = hid_dim
        self.out_dim = out_dim
        self.layer = num_layers
        
        self.gru = nn.GRU(self.in_dim, self.hid_dim, self.layer, batch_first=True)
        self.fc = nn.Linear(self.hid_dim, self.out_dim)
        
    def forward(self,x):
        
        # initalize the hidden layers
        h0 = torch.zeros(self.layer, x.size(0), self.hid_dim)
        
        out, _ = self.gru(x,h0)
        out = self.fc(out[:,-1,:])
        return out

In [ ]:
# convert the data from numpy to tensor
train_x2, train_y2, test_x2, test_y2 = split_data(x, y, 0.7)
train_x2 = torch.from_numpy(train_x2.astype(np.float32))
train_y2 = torch.from_numpy(train_y2.astype(np.float32))
test_x2 = torch.from_numpy(test_x2.astype(np.float32))
test_y2 = torch.from_numpy(test_y2.astype(np.float32))

In [ ]:
# definelGRU neurak network
in_size = x.shape[-1]
out_size = in_size
hid_size = 5
num_layers = 1

In [ ]:
#__________ GRU Model__________________________________
gru = GRU(in_size,hid_size,out_size, num_layers)
optimizer = torch.optim.Adam(gru.parameters(), lr=0.01)
loss_fun = nn.MSELoss()

In [ ]:
epochs2=500
train_loss2, test_loss2 = train_model(gru,
               loss_fun,
               optimizer,
               train_x2,
               test_x2,
               train_y2,
               test_y2,
               epochs=epochs2)

In [ ]:
trace13 = go.Scatter(
    x = np.arange(0, epochs1,1),
    y = train_loss2,
    mode = 'lines',
    name = 'Train Loss'
)
trace23 = go.Scatter(
    x = np.arange(0, epochs1,1),
    y = test_loss2,
    mode = 'lines',
    name = 'Test Loss'
)

layout = go.Layout(
    title = f"RNN performace with GRU model",
    xaxis = {'title' : 'Epochs'},
    yaxis = {'title' : 'Loss'}
)
fig = go.Figure(data=[trace13, trace23], layout=layout)
fig.show()

In [ ]:
# testing the prediction quality if the trained model

trace33 = go.Scatter(
    y = np.vstack((train_y2.detach().numpy(),test_y2.detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Target'
)
trace43 = go.Scatter(
    y = np.vstack((train_y2.detach().numpy(),gru(test_x2).detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Prediction'
)

layout = go.Layout(
    title = f"Evolution of feature '{target}' with time",
    xaxis = {'title' : 'Time delta in days'},
    yaxis = {'title' : f'{target}'}
)
fig = go.Figure(data=[trace33, trace43], layout=layout)
fig.show()

### Autoregressive Model

In [ ]:
# convert the data from numpy to tensor
train_x3, train_y3, test_x3, test_y3 = split_data(x, y, 0.7)
train_x3 = torch.from_numpy(train_x3.astype(np.float32))
train_y3 = torch.from_numpy(train_y3.astype(np.float32))
test_x3 = torch.from_numpy(test_x3.astype(np.float32))
test_y3 = torch.from_numpy(test_y3.astype(np.float32))

In [ ]:
# Implement Autoregressive model
auto_reg = nn.Linear(10,1)
optimizer = torch.optim.Adam(auto_reg.parameters(), lr=0.01)
loss_fun = nn.MSELoss()

In [ ]:
epochs3=500
train_loss3, test_loss3 = train_model(auto_reg,
               loss_fun,
               optimizer,
               train_x3.reshape(-1,10),
               test_x3.reshape(-1,10),
               train_y3.reshape(-1,1),
               test_y3.reshape(-1,1),
               epochs=epochs3)

In [ ]:
trace14 = go.Scatter(
    x = np.arange(0, epochs1,1),
    y = train_loss3,
    mode = 'lines',
    name = 'Train Loss'
)
trace24 = go.Scatter(
    x = np.arange(0, epochs1,1),
    y = test_loss3,
    mode = 'lines',
    name = 'Test Loss'
)

layout = go.Layout(
    title = f"RNN performace with Autorgressive model",
    xaxis = {'title' : 'Epochs'},
    yaxis = {'title' : 'Loss'}
)
fig = go.Figure(data=[trace14, trace24], layout=layout)
fig.show()

In [ ]:
# testing the prediction quality if the trained model

trace34 = go.Scatter(
    y = np.vstack((train_y3.detach().numpy(),test_y3.detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Target'
)
trace44 = go.Scatter(
    y = np.vstack((train_y3.detach().numpy(),auto_reg(test_x3.reshape(-1,10)).detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Prediction'
)

layout = go.Layout(
    title = f"Evolution of feature '{target}' with time",
    xaxis = {'title' : 'Time delta in days'},
    yaxis = {'title' : f'{target}'}
)
fig = go.Figure(data=[trace34, trace44], layout=layout)
fig.show()

In [ ]:
# testing the prediction for all models

trace15 = go.Scatter(
    y = np.vstack((train_y.detach().numpy(),test_y.detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Target data'
)
trace25 = go.Scatter(
    y = np.vstack((train_y.detach().numpy(),model(test_x).detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Prediction with simple RNN'
)

trace35 = go.Scatter(
    y = np.vstack((train_y1.detach().numpy(),lstm(test_x1).detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Prediction with LSTM model'

)
trace45 = go.Scatter(
    y = np.vstack((train_y2.detach().numpy(),gru(test_x2).detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Prediction with GRU model'
)

trace55 = go.Scatter(
    y = np.vstack((train_y3.detach().numpy(),auto_reg(test_x3.reshape(-1,10)).detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Prediction with Autoregressive Model'
)
layout = go.Layout(
    title = f"Evolution of feature '{target}' with time",
    xaxis = {'title' : 'Time delta in days'},
    yaxis = {'title' : f'{target}'}
)
fig = go.Figure(data=[trace15, trace25, trace35, trace45, trace55], layout=layout)
fig.update_layout(
    autosize=False,
    width=1200,
    height=800
    )
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.show()